In [1]:
import numpy as np
import numpy.matlib as nm
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform

class SVGD():

    def __init__(self):
        pass
    
    def svgd_kernel(self, theta, model, h = -1):
        if theta.shape[0]>1:
            sq_dist = pdist(theta)
            pairwise_dists = squareform(sq_dist)**2
            if h < 0: # if h < 0, using median trick
                h = np.median(pairwise_dists)  
                h = np.sqrt(0.5 * h / np.log(theta.shape[0]+1))

            # compute the rbf kernel
            w_s = model.cal_w_s(theta)
            Kxy = np.exp( -pairwise_dists / h**2 / 2) 
            wxK = nm.repmat(w_s[np.newaxis,:], Kxy.shape[0], 1)   #  every xj's weights 
#             print('wxK:',wxK)
            Kxy = np.multiply(Kxy, wxK)     # weighted kernel
            dxkxy = -np.matmul(Kxy, theta)
            sumkxy = np.sum(Kxy, axis=1)
            for i in range(theta.shape[1]):
                dxkxy[:, i] = dxkxy[:,i] + np.multiply(theta[:,i],sumkxy)
            dxkxy = dxkxy / (h**2)
            
        else:
            Kxy = np.ones((1,1))
            dxkxy = np.zeros(theta.shape)
        return (Kxy, dxkxy)
    
 
    def update(self, model, x0, n_iter = 1000, stepsize = 1e-3, bandwidth = -1, alpha = 0.9, debug = False):
        # Check input
        if x0 is None or model.dlnprob is None:
            raise ValueError('x0 or lnprob cannot be None!')
        
        theta = np.copy(x0) 
        
        # adagrad with momentum
        fudge_factor = 1e-6
        historical_grad = 0
        for iter in range(n_iter):
            if debug and (iter+1) % 1000 == 0:
                print ('iter ' + str(iter+1))
            # lamda = 1 - iter/(2*n_iter)
            # model.anneal(lamda)
            # show1(theta, model, iter)
#             print('theta:',theta)
            lnpgrad = model.dlnprob(theta)
            # calculating the kernel matrix
            kxy, dxkxy = self.svgd_kernel(theta, model, h = -1)  
            grad_theta = (np.matmul(kxy, lnpgrad) + dxkxy) / x0.shape[0]  
            
            # adagrad 
            if iter == 0:
                historical_grad = historical_grad + grad_theta ** 2
            else:
                historical_grad = alpha * historical_grad + (1 - alpha) * (grad_theta ** 2)
            adj_grad = np.divide(grad_theta, fudge_factor+np.sqrt(historical_grad))
            theta = theta + stepsize * adj_grad 
            
        return theta

In [2]:
import numpy as np
import numpy.matlib as nm
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from numpy.linalg import inv, eig
import scipy.io as scio

def get_prob(x0, idx, model):
    x0[idx] = 1
    p1 = np.exp(np.matmul(np.matmul(x0, model.A), x0.T) + np.matmul(x0,model.B))
    x0[idx] = -1
    p_1 = np.exp(np.matmul(np.matmul(x0, model.A), x0.T) + np.matmul(x0,model.B))
    p = p1/(p1+p_1)
    return p

def gibbs(n, model, dim):
    samples = np.zeros((n+50, dim))
    
    x0 = np.random.binomial(1, np.random.rand(1)[0], dim)
    x0[np.where(x0==0)] =  -1

    for i in range(n+50):
        for idx in range(dim):
            p = get_prob(x0, idx, model)
            x = np.random.binomial(1, p, 1) # need to convert 0, 1 to -1, 1
            x0[idx] = 2*x[0]-1
        samples[i, :] = x0

    return samples[50:,:]

class MVN:
    def __init__(self, A, B):
        self.A = A
        self.B = B
        value = self.gen_value()
        pdf = np.ones(value.shape[0])
        for i in range(value.shape[0]):
            pdf[i] = np.exp(np.matmul(np.matmul(value[i,:], self.A), value[i,:].T) + np.matmul(value[i,:],self.B))
        self.pTrue = pdf/pdf.sum()
        self.muTrue = np.sum(np.multiply(value, nm.repmat(self.pTrue[:,np.newaxis], 1, value.shape[1])), 0)
        self.p = 1 - np.sum(pdf[0:512])/pdf.sum()
    
    def dlnprob(self, theta):
        return  2*np.matmul(theta, self.A) + self.B - theta
    
    def anneal(self,lamda):
        pass
    
    def cal_w_s(self, x0):
        w_s = np.ones(x0.shape[0])
        for i in range(x0.shape[0]):
            phat = np.exp(np.matmul(np.matmul(x0[i,:], self.A), x0[i,:].T) + np.matmul(x0[i,:],self.B))
            p    = np.exp(np.matmul(np.matmul(np.sign(x0[i,:]), self.A), np.sign(x0[i,:]).T) + np.matmul(np.sign(x0[i,:]),self.B))
            w_s[i] = 1.0*phat/p

        w_s = (w_s/np.sum(w_s)).squeeze()
        return w_s

    def gen_value(self):
        dim = self.B.shape[0]
        value = []
        def ten2two(x):
            a=[]
            for i in bin(x)[2:]:
                a.append(2*int(i)-1)
            a = [ -1 for _ in range(dim - len(a)) ] + a
            return np.array(a)

        for k in range(int(np.power(2,dim))):
            value.append(ten2two(k))
        return np.array(value)

In [ ]:
if __name__ == '__main__':
    dim = 10
    limit = 0.15 # pairwise stength limit
    strength = np.linspace(-limit,limit,31)
    n = 20 #samples
    B = np.ones(dim)*0.01

    ising = []
    ising2 = []
    ising3 = []
    
    for strong in range(len(strength)):
        A = (np.ones((dim,dim))- np.diag(np.ones(dim)))*strength[strong]/2
        model = MVN(A,B)
        print('turn=',strong)
        print('    pTrue=',model.p)
        print('    muTrue=',model.muTrue[0])
        # GF-SVGD
        theta1 = 0
        for j in range(10):
            x0 = np.random.normal(0,1,[n,dim])
            theta = SVGD().update(model, x0, n_iter=2000, stepsize=0.01)
            theta1 += np.mean((np.mean(theta,0)-model.muTrue)**2)/10

        ising.append(theta1)
        # Monte Carlo
        theta2 = 0
        for j in range(1000):
            for k in range(dim):
                theta = np.random.binomial(1, model.p, n)
                theta[np.where(theta==0)]=-1
                theta2 += ((np.mean(theta)-model.muTrue[k])**2)/(1000*dim)

        ising2.append(theta2)
        # Gibbs Sampling
        theta3 = 0
        for j in range(1000):
            theta = gibbs(n, model, dim)
            theta3 += np.mean((np.mean(theta,0)-model.muTrue)**2)/1000

        ising3.append(theta3)

        print('        GF-SVGD',theta1)
        print('        Monte-Carlo',theta2)
        print('        Gibbs',theta3)
    
    ising = np.log10(np.array(ising))
    ising2 = np.log10(np.array(ising2))
    ising3 = np.log10(np.array(ising3))
    scio.savemat('fig/ising.mat', {'ising':ising})
    scio.savemat('fig/ising2.mat', {'ising2':ising2})
    scio.savemat('fig/ising3.mat', {'ising3':ising3})
    # ising = scio.loadmat('ising.mat')['ising'].squeeze()
    # ising2 = scio.loadmat('ising2.mat')['ising2'].squeeze()
    # ising3 = scio.loadmat('ising3.mat')['ising3'].squeeze()
    plt.figure(figsize=(8,6))
    x = strength
    plt.plot(x, ising2, 'r-s', label='Exact Monte Carlo',linewidth=3, markersize=18)
    plt.plot(x, ising3, 'b-D', label='Gibbs Sampling',linewidth=3, markersize=18)
    plt.plot(x, ising, 'g-o', label='Gradient Free SVGD',linewidth=3, markersize=18)
    plt.xlabel('Pairwise Strength',fontsize=30)
    plt.ylabel('log10 MSE',fontsize=30)
    # plt.xlim([1,7])
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.legend(fontsize=16)
#     plt.savefig('strength.pdf',format='pdf')
    plt.show()

turn= 0
    pTrue= 0.502049067473
    muTrue= 0.00409813494606
        GF-SVGD 0.0111411315222
        Monte-Carlo 0.0507149279431
        Gibbs 0.0555055833694
turn= 1
    pTrue= 0.50213520795
    muTrue= 0.00427041589936
        GF-SVGD 0.0350678441347
        Monte-Carlo 0.0498719413989
        Gibbs 0.0563895430018
turn= 2
    pTrue= 0.502228761594
    muTrue= 0.00445752318717
        GF-SVGD 0.00784171044044
        Monte-Carlo 0.0500819923367
        Gibbs 0.0548089915171
turn= 3
    pTrue= 0.502330704695
    muTrue= 0.00466140938905
        GF-SVGD 0.00408418148487
        Monte-Carlo 0.0505111304154
        Gibbs 0.054639742657
turn= 4
    pTrue= 0.502442187539
    muTrue= 0.00488437507862
        GF-SVGD 0.00214059404099
        Monte-Carlo 0.0500723885564
        Gibbs 0.0550888000567
turn= 5
    pTrue= 0.502564573463
    muTrue= 0.0051291469263
        GF-SVGD 0.00276868497588
        Monte-Carlo 0.0503414004467
        Gibbs 0.0540776602098
turn= 6
    pTrue= 0.50269948836
